In [1]:
import gensim

In [11]:
import pandas as pd
import numpy as np

root_path = r'C:\Users\bened\DataScience\ANLP\AT2'
df = pd.read_csv(f'{root_path}\\36118_NLP_Spring\CSVs\df_aus_preprocessed2.csv')
df.head()

,Unnamed: 0,imdbid,title,year,opening weekend,budget,age restrict,genres,screenplay,age restrict aus,screenplay_sentences,clean_sentences,script,clean_scripts,stemmed_scripts,processed_scripts
0,0,120770,A Night at the Roxbury,1998,United States:,"$17,000,000 (estimated)","Argentina:13, Australia:M, Brazil:14, Canada:P...","Comedy, Music, Romance",\n\n\t\t\t A NIGHT AT THE ROXBURY \n\n\n\t\...,M,['\n\n\t\t\t A NIGHT AT THE ROXBURY \n\n\n\...,['night roxbury written steve koren ferrell ch...,"['a night at the roxbury', 'written by', 'stev...","['night roxbury', 'written', 'steve koren', 'f...","['night roxburi', 'written', 'steve koren', 'f...",night roxburi\nwritten\nsteve koren\nferrel\nc...
1,1,132512,At First Sight,1999,United States:,"$60,000,000 (estimated)","Argentina:13, Australia:M, Canada:PG::(Alberta...","Drama, Romance",AT FIRST SIGHT\n\nEXT. VALLEY - DUSK \nGold li...,M,"['AT FIRST SIGHT\n\nEXT.', 'VALLEY - DUSK \nGo...","['first sight ext', 'valley dusk gold light da...","['at first sight', 'ext valley dusk', 'gold li...","['first sight', 'ext valley dusk', 'gold light...","['first sight', 'ext valley dusk', 'gold light...",first sight\next valley dusk\ngold light dappl...
2,2,118661,The Avengers,1998,"United States: $10,305,957, 16 Aug 1998","$60,000,000 (estimated)","Argentina:13, Australia:PG, Brazil:10, Canada:...","Action, Adventure, Sci-Fi, Thriller",\n\n\t\t\t\t\tTHE AVENGERS\n\n\t\t\t\tScreenpl...,PG,['\n\n\t\t\t\t\tTHE AVENGERS\n\n\t\t\t\tScreen...,['avengers screenplay macpherson june 21 1995 ...,"['the avengers', 'screenplay by don macpherson...","['avengers', 'screenplay macpherson', 'june 21...","['aveng', 'screenplay macpherson', 'june 21 19...",aveng\nscreenplay macpherson\njune 21 1995\n1b...
3,4,118715,The Big Lebowski,1998,"United States: $5,533,844, 08 Mar 1998","$15,000,000 (estimated)","Argentina:16, Argentina:18::(cable rating), Au...","Comedy, Crime, Sport",\n\n\t\t\tTHE BIG LEBOWSKI\n\nWe are floating ...,MA,['\n\n\t\t\tTHE BIG LEBOWSKI\n\nWe are floatin...,"['big lebowski floating steep scrubby slope', ...","['the big lebowski', 'we are floating up a ste...","['big lebowski', 'floating steep scrubby slope...","['big lebowski', 'float steep scrubbi slope he...",big lebowski\nfloat steep scrubbi slope hear m...
4,5,112571,Boys on the Side,1995,NaN,NaN,"Argentina:13, Australia:MA, Canada:14A::(Manit...","Comedy, Drama",Boys on the Side\n\nSCENE 1\n\nJANE\nThank you...,MA,['Boys on the Side\n\nSCENE 1\n\nJANE\nThank y...,"['boys side scene 1 jane thank', 'jane deluca ...","['boys on the side', 'scene 1', 'thank you i m...","['boys side', 'scene 1', 'thank jane deluca jo...","['boy side', 'scene 1', 'thank jane deluca joh...",boy side\nscene 1\nthank jane deluca johnni fi...


In [12]:
# parse stemmed_scripts as lists of sentences 
import ast 

def back_to_list(string):
    sents = ast.literal_eval(string)
    return sents

sentences = df['stemmed_scripts'].apply(back_to_list)
print(sentences[0])

['night roxburi', 'written', 'steve koren', 'ferrel', 'chris kattan', 'june 2 1997', 'fade', 'ext panoram view los angel sunset', 'hear love haddaway night fall partytim begin', 'superimpos sunset blvd 11 03 pm', 'cut', 'ext danc club night', 'coconut teaser palac roxburi tatou etc', 'cut', 'int danc club quick shot night', 'random dancer gyrat flirt make drink', 'cut', 'int palac night', 'camera move crowd danc floor settl rhythmic sway back', 'steve doug butabi', 'hero mind steve tall dark handsom doug littl genius neither correct except tall littl part', 'simultan turn scope room unison head bop music doug step bar', 'femal', 'hey want danc', 'yes alright worri', 'doug reject step back steve step', 'femal', 'want danc', 'problem', 'stranger reject neither faze doug enthusiast step toward two attract girl', 'hey wanna', 'two attract girl turn back doug', 'remain posit', 'mayb see later', 'doug step back steve spot girl end bar danc', 'hey want danc', 'girl bar', 'alright know', 'stev

In [13]:
sentences

0       [night roxburi, written, steve koren, ferrel, ...
1       [first sight, ext valley dusk, gold light dapp...
2       [aveng, screenplay macpherson, june 21 1995, 1...
3       [big lebowski, float steep scrubbi slope hear ...
4       [boy side, scene 1, thank jane deluca johnni f...
                              ...                        
1886    [vo ud, twenti pirst centuri por trale, wreete...
1887    [june 29 1994, sleep, dan sullivan fred lebow,...
1888    [wind river, written, taylor sheridan, open, s...
1889    [wrong accus, pat proft, proft thing intern re...
1890    [screenplay lynn ramsay, base novella jonathan...
Name: stemmed_scripts, Length: 1891, dtype: object

In [7]:
from gensim.models import phrases, word2vec

# streams preprocessed_screenplays into the Phrases model 

texts_path = r'C:\Users\bened\DataScience\ANLP\AT2\preprocessed_screenplays\\'

phrases = phrases.Phrases(
    sentences=word2vec.PathLineSentences(texts_path),
    connector_words=phrases.ENGLISH_CONNECTOR_WORDS
)

phrase_dict = phrases.export_phrases()
phrase_dict

{'tire_squeal': 74.79036534147156,
 'split_second': 47.45694939165933,
 'glass_shatter': 17.467883607785524,
 'smash_cut': 11.34119069867137,
 'dim_light': 24.95409775265144,
 'light_flicker': 19.982992394313765,
 'fluoresc_light': 65.76978388360467,
 'light_illumin': 13.824032998045427,
 'young_woman': 19.092536010209972,
 'mattress_floor': 10.694524191961952,
 'mid_western': 24.319901268761335,
 'bare_conscious': 41.60811948658359,
 'muffl_thump': 21.25228930307942,
 'blood_soak': 46.64129947837897,
 'cinder_block': 487.0372669618284,
 'cement_floor': 22.158521237082677,
 'eye_flit': 18.193260975454482,
 'sear_pain': 29.377570216356602,
 'cell_phone': 58.08350338857343,
 'take_deep': 12.493665603181313,
 'deep_breath': 86.94629418637678,
 'grit_teeth': 1031.4760286853389,
 'doesnâ_budg': 63.22781720149555,
 'basebal_bat': 364.78131912713616,
 'creak_open': 13.44919459394051,
 'tray_food': 37.77877679836095,
 'latch_onto': 27.73310048901713,
 'blood_seep': 117.54264532862884,
 'grip_t

In [8]:
import pandas as pd

phrase_df = pd.DataFrame.from_dict(
    phrase_dict, 
    orient='index'
)
phrase_df.reset_index(inplace=True)
phrase_df.columns = ["phrase", "score"]
phrase_df.sort_values('score', inplace=True, ascending=False)
print(phrase_df.head(10))
print(phrase_df.tail(10))

                                phrase          score
15248                    no_fullscreen  684138.913043
22476                             it_i  503526.240000
15639                 continue_for_fre  370239.882353
11043                            i_hav  328386.678261
15646                  startpag_endpag  310818.666667
38663                   mittag_leffler  310818.666667
27392                 kwisatz_haderach  295034.906250
35626                     minato_mirai  282426.576923
31960                  loftus_versfeld  282426.576923
15555  extralight_sofia_pro_extralight  279736.800000
               phrase      score
9652      uncl_arthur  10.010101
13829      dick_clark  10.009793
1077         song_end  10.009597
2859   schofieldâ_eye  10.008759
6705   overweight_man  10.006280
11773       bystand_1  10.003833
20444       nerdi_guy  10.003258
23004     talent_show  10.002933
13144     happi_oblig  10.002259
29998    junior_senat  10.001602


TODO: 

- check for consistency between numerical integers and lexical numbers, e.g. should "10" and "ten" be made the same? 

In [9]:
frozen_phrases = phrases.freeze()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\bened\\DataScience\\ANLP\\AT2\\models\\tmp\\phrase_model.pkl'

In [10]:
# save_path = r'C:\Users\bened\DataScience\ANLP\AT2\models'
# frozen_phrases.save(f"{save_path}\\phrase_model.pkl")

## Continuous Bag of Words (CBOW)

In [ ]:
sents = sentences.to_list()
sents

In [ ]:

cbow_model = Word2Vec(
    sentences=frozen_phrases[sentences],